<h1>Data Handling - Preparing Data for Modelling</h1>

<h2>Preliminary Steps</h2>

Let's begin with importing the necessary libraries:

In [2]:
import pandas as pd
import scipy as sc
import numpy as np
import math
import pickle

import warnings
warnings.filterwarnings("ignore")

For this step, we need to load the dataframe which is the final product of our three crawling steps:

In [3]:
df_original = pd.read_csv("df.csv")

<h2>The Process</h2>

<h3>Data Analysis</h3>

In [4]:
df_original.head()

,weekday,day,month,year,time,aircraft_type,num_of_engines,engine_type,engine_model,years_active,...,operator,occupants,accident_loc,above_ocean,flight_phase,damage,fate,accident_latitude,accident_longtitude,fatalities
0,Saturday,2.0,August,1919,NaN,Caproni Ca.48,3.0,piston,Liberty L-12,0.00,...,Caproni,14.0,Italy,0,ENR,Destroyed,Written off,45.396389,10.888056,14.0
1,Monday,11.0,August,1919,NaN,Felixstowe Fury,5.0,piston,Rolls-Royce Eagle VIII,0.75,...,Royal Air Force - RAF,7.0,United Kingdom,0,ICL,Damaged beyond repair,NaN,51.941370,1.306789,1.0
2,Monday,23.0,February,1920,NaN,Handley Page Type O,2.0,piston,NaN,1.00,...,Handley Page Transport,10.0,South Africa,0,ENR,Damaged beyond repair,NaN,NaN,NaN,0.0
3,Wednesday,25.0,February,1920,NaN,Handley Page Type O,2.0,piston,NaN,NaN,...,Handley Page Transport,4.0,Sudan,0,UNK,Damaged beyond repair,NaN,NaN,NaN,0.0
4,Wednesday,30.0,June,1920,NaN,Handley Page Type O,2.0,piston,NaN,1.00,...,Handley Page Transport,2.0,Sweden,0,ENR,Damaged beyond repair,NaN,NaN,NaN,0.0


We need to analyze the dataframe we've scraped. Let's get familliar with what are the columns we built:<br>
<ul>
    <li><b>weekday</b> - Specifies the day of the week when the accident has occurred.</li>
    <li><b>day</b> - Specifies the day of the month when the accident has occurred.</li>
    <li><b>month</b> - Specifies the Gregorian-calendar month when the accident has occurred.</li>
    <li><b>year</b> - Specifies the year when the accident has occurred.</li>
    <li><b>time</b> - Specifies the time of the day when the accident has occured, as a float number on the sequence between 0 to 24.</li>
    <li><b>aircraft_type</b> - Specifies the model of the aircraft involved in the accident.</li>
    <li><b>num_of_engines</b> - Specifies the amount of engines the aircraft involved in the accident has.</li>
    <li><b>engine_type</b> - Specifies the engine type of the aircraft involved in the accident.</li>
    <li><b>engine_model</b> - Specifies the model of the engine of the aircraft involved in the accident.</li>
    <li><b>years_active</b> - Specifies the time passed since the aircraft involved in the accident had its first flight.</li>
    <li><b>airframe_hrs</b> - Specifies the total amount of hours the aircraft spent airborne until the accident.</li>
    <li><b>cycles</b> - Specifies the total number of flights the aircraft had until the accident.</li>
    <li><b>operator</b> - Specifies the operating airline/management of the aircraft at the time of the accident.</li>
    <li><b>occupants</b> - Specifies the number of passengers and crew members that were present onboard the aircraft at the time of the accident.</li>
    <li><b>accident_loc</b> - Specifies the location where the accident occurred (if was above land, specifies the country or territory. Otherwise, specifies the name of the body of water).</li>
    <li><b>above_ocean</b> - Specifies whether the aircraft was above ocean when the accident occurred.</li>
    <li><b>flight_phase</b> - Specifies in which phase of the flight the accident has occurred</li>
    <li><b>damage</b> - Specifies the damage the aircraft has sufferred following the accident.</li>
    <li><b>fate</b> - Specifies what happened to the aircraft following the accident.</li>
    <li><b>accident_latitude</b> and <b>accident_longtitude</b> - Specifies the coordinates of the accident's location.</li>
    <li><b>fatalities</b> - Specifies the number of occupants that were killed as a result of the accident.</li>
</ul>

In [5]:
df.describe(include='all')

NameError: name 'df' is not defined

<h3>Handling Missing Data</h3>

Now, we need to analyze the dataframe in order to see whether there are and how many values missing in the dataframe as a whole, and in every column individually

In [6]:
df_original.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22220 entries, 0 to 22219
Data columns (total 22 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   weekday              21736 non-null  object 
 1   day                  21736 non-null  float64
 2   month                21736 non-null  object 
 3   year                 22220 non-null  int64  
 4   time                 8504 non-null   float64
 5   aircraft_type        22187 non-null  object 
 6   num_of_engines       22112 non-null  float64
 7   engine_type          22112 non-null  object 
 8   engine_model         10063 non-null  object 
 9   years_active         16827 non-null  float64
 10  airframe_hrs         4152 non-null   float64
 11  cycles               1502 non-null   float64
 12  operator             21580 non-null  object 
 13  occupants            16290 non-null  float64
 14  accident_loc         22204 non-null  object 
 15  above_ocean          22220 non-null 

We have two methods to handle missing data: Filling it with the frequent/median/mean value of the column or to remove it.<br><br>
While ideally we would like to use the filling method to perserve and maximize the amount of records/columns in our data, it's not always possible and our data may become biased. To avoid data biasing, we need to examine every column and decide:<br>
<ul>
    <li>Whether it is important enough to keep</li>
    <li>If it is, whether the data can be filled without degrading from its genuinity</li>
    <li>If not, remove the record rows with missing values in that column</li>
</ul><br>
This process will drastically reduce the remaining data, but it will be ready for the visualization and modelling steps

<h4>Removing Cells</h4>

<b>1)</b> We will create a copy of our dataframe to monitor the changes we make to it better

In [7]:
df = df_original

<b>2)</b> The first columns we know we would like to keep, but many records do not have assigned data to it are:<br>
<b>'fatalities'</b> - this is a very important column, as we plan to make it our target column in the next steps. Therefore it must not lack data.<br>
<b>'accident_latitude'</b> and <b>'accident_longtitude'</b> - As they are sequential variables, and we handled their scraping diffrently than other columns.<br>
<b>'weekday', 'day' and month'</b> - Not too many rows lack them, and filling them might be tricky as the weekday depends on the the day, month and the year.<br><br>
These columns cannot have their values filled. Therefore, we would first remove all rows that miss values in these columns:

In [8]:
df.dropna(axis=0, subset=['accident_latitude','accident_longtitude','fatalities','weekday','day','month'], inplace=True) 
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7691 entries, 0 to 22215
Data columns (total 22 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   weekday              7691 non-null   object 
 1   day                  7691 non-null   float64
 2   month                7691 non-null   object 
 3   year                 7691 non-null   int64  
 4   time                 5378 non-null   float64
 5   aircraft_type        7670 non-null   object 
 6   num_of_engines       7649 non-null   float64
 7   engine_type          7649 non-null   object 
 8   engine_model         4883 non-null   object 
 9   years_active         6751 non-null   float64
 10  airframe_hrs         2859 non-null   float64
 11  cycles               1129 non-null   float64
 12  operator             7292 non-null   object 
 13  occupants            7415 non-null   float64
 14  accident_loc         7691 non-null   object 
 15  above_ocean          7691 non-null   

<b>3)</b> Let's see which rows still have missing data and examine whether we can fill them up or not:<br>
<b>'engine_model</b> - A categorical column, which can be filled only with the most frequent value, but might not represent an engine model that is appropriate for certain aircraft types.

In [9]:
df.dropna(axis=0, subset=['engine_model'], inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4883 entries, 0 to 22213
Data columns (total 22 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   weekday              4883 non-null   object 
 1   day                  4883 non-null   float64
 2   month                4883 non-null   object 
 3   year                 4883 non-null   int64  
 4   time                 3916 non-null   float64
 5   aircraft_type        4865 non-null   object 
 6   num_of_engines       4856 non-null   float64
 7   engine_type          4856 non-null   object 
 8   engine_model         4883 non-null   object 
 9   years_active         4649 non-null   float64
 10  airframe_hrs         2391 non-null   float64
 11  cycles               965 non-null    float64
 12  operator             4557 non-null   object 
 13  occupants            4745 non-null   float64
 14  accident_loc         4883 non-null   object 
 15  above_ocean          4883 non-null   

<b>'engine_type'</b>, <b>'num_of_engines'</b> and <b>'aircraft_type</b> - Those columns values' are a direct derivative of the 'aircraft_type' column, which means that traditional methods of filling missing data would not provide accurate resuls most of the time.<br>
<b>'operator'</b> - Filling the missing operators with the most frquent operator will bias the modelling chances in favor to this operator.<br>
Therefore, we will remove all rows missing them as well:

In [10]:
df.dropna(axis=0, subset=['engine_type', 'num_of_engines', 'aircraft_type', 'operator'], inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4531 entries, 0 to 22213
Data columns (total 22 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   weekday              4531 non-null   object 
 1   day                  4531 non-null   float64
 2   month                4531 non-null   object 
 3   year                 4531 non-null   int64  
 4   time                 3608 non-null   float64
 5   aircraft_type        4531 non-null   object 
 6   num_of_engines       4531 non-null   float64
 7   engine_type          4531 non-null   object 
 8   engine_model         4531 non-null   object 
 9   years_active         4298 non-null   float64
 10  airframe_hrs         2189 non-null   float64
 11  cycles               876 non-null    float64
 12  operator             4531 non-null   object 
 13  occupants            4399 non-null   float64
 14  accident_loc         4531 non-null   object 
 15  above_ocean          4531 non-null   

<b>4)</b> The remaining missing data in most of the columns might now will be able to be filled, however there are some columns that even after removing rows, still keep a large gap of missing data compared to the remaining number of entries. Let's review them and decide what can be done with them:<br>

In [11]:
df.describe(include='all')

,weekday,day,month,year,time,aircraft_type,num_of_engines,engine_type,engine_model,years_active,...,operator,occupants,accident_loc,above_ocean,flight_phase,damage,fate,accident_latitude,accident_longtitude,fatalities
count,4531,4531.000000,4531,4531.000000,3608.000000,4531,4531.000000,4531,4531,4298.000000,...,4531,4399.000000,4531,4531.000000,4531,4531,2500,4531.000000,4531.000000,4531.000000
unique,7,NaN,12,NaN,NaN,309,NaN,3,123,NaN,...,2342,NaN,198,NaN,10,7,4,NaN,NaN,NaN
top,Wednesday,NaN,January,NaN,NaN,Douglas DC-3,NaN,jet,Pratt & Whitney Canada PT6,NaN,...,United States Air Force - USAF,NaN,United States of America,NaN,LDG,Damaged beyond repair,Written off,NaN,NaN,NaN
freq,715,NaN,444,NaN,NaN,342,NaN,2072,641,NaN,...,153,NaN,1141,NaN,1336,1836,1836,NaN,NaN,NaN
mean,NaN,15.728537,NaN,1991.058707,13.395094,NaN,2.438314,NaN,NaN,16.360400,...,NaN,40.919300,NaN,0.016773,NaN,NaN,NaN,27.578006,-13.247964,13.646877
std,NaN,8.806055,NaN,20.693689,5.639869,NaN,0.896339,NaN,NaN,13.136456,...,NaN,61.455952,NaN,0.128435,NaN,NaN,NaN,24.694867,80.857836,33.506538
min,NaN,1.000000,NaN,1919.000000,0.016667,NaN,1.000000,NaN,NaN,0.000000,...,NaN,0.000000,NaN,0.000000,NaN,NaN,NaN,-83.920000,-179.868000,0.000000
25%,NaN,8.000000,NaN,1976.000000,9.366667,NaN,2.000000,NaN,NaN,6.000000,...,NaN,4.000000,NaN,0.000000,NaN,NaN,NaN,10.765318,-80.259063,0.000000
50%,NaN,16.000000,NaN,1996.000000,13.566667,NaN,2.000000,NaN,NaN,14.000000,...,NaN,12.000000,NaN,0.000000,NaN,NaN,NaN,34.566500,-2.776605,2.000000
75%,NaN,23.000000,NaN,2008.000000,17.916667,NaN,3.000000,NaN,NaN,24.000000,...,NaN,52.000000,NaN,0.000000,NaN,NaN,NaN,45.271517,37.842913,9.000000


<b>'cycles'</b> - While might be useful, it lacks data for too many entries, and using the mean to fill the missing data in it will bias the dataframe. Therfore, it will be deleted.<br>
<b>'fate' and 'airframe_hrs'</b> - Both lack roughly the same amount of data. However, 'airframe_hrs' is a sequential column that its values are derived from other sequential columns, so few calculations may help to fill its missing values.<br>
'fate' on the other hand is a categorical column, with 'Written off' as the frquent value, with 1836 appearances. Filling 2500 missing values with a value that appears less times doesn't sound reflectng.<br><br>
Therefore, the 'cycles' and 'fate' columns will be removed from the dataframe:

In [12]:
df.drop(['cycles', 'fate'], axis=1, inplace=True)
df.reset_index(drop=True, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4531 entries, 0 to 4530
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   weekday              4531 non-null   object 
 1   day                  4531 non-null   float64
 2   month                4531 non-null   object 
 3   year                 4531 non-null   int64  
 4   time                 3608 non-null   float64
 5   aircraft_type        4531 non-null   object 
 6   num_of_engines       4531 non-null   float64
 7   engine_type          4531 non-null   object 
 8   engine_model         4531 non-null   object 
 9   years_active         4298 non-null   float64
 10  airframe_hrs         2189 non-null   float64
 11  operator             4531 non-null   object 
 12  occupants            4399 non-null   float64
 13  accident_loc         4531 non-null   object 
 14  above_ocean          4531 non-null   int64  
 15  flight_phase         4531 non-null   o

<h4>Filling Missing Cells</h4>

From the remaining 20 columns and 4531 entries, we still have the following four columns that still have missing data:
<ul>
    <li>'time'</li>
    <li>'years_active'</li>
    <li>'airframe_hrs'</li>
    <li>'occupants'</li>
</ul><br>We'll now fill the missing data in these columns in various ways:

<b>1) 'time'</b> - filling it will be straightforward, with the median time of all other entries:

In [13]:
df['time'].fillna(df['time'].median(), inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4531 entries, 0 to 4530
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   weekday              4531 non-null   object 
 1   day                  4531 non-null   float64
 2   month                4531 non-null   object 
 3   year                 4531 non-null   int64  
 4   time                 4531 non-null   float64
 5   aircraft_type        4531 non-null   object 
 6   num_of_engines       4531 non-null   float64
 7   engine_type          4531 non-null   object 
 8   engine_model         4531 non-null   object 
 9   years_active         4298 non-null   float64
 10  airframe_hrs         2189 non-null   float64
 11  operator             4531 non-null   object 
 12  occupants            4399 non-null   float64
 13  accident_loc         4531 non-null   object 
 14  above_ocean          4531 non-null   int64  
 15  flight_phase         4531 non-null   o

<b>2) 'occupants'</b> -  Since values in the 'fatalities' column cannot be higher than the respective values in 'occupants', and we now don't have missing values in the 'fatalities' column, we will use two approaches:<br>
In a certain row with missing 'occupants value:
<ol>
    <li>If the number of fatalities is higher than the 'occupants' average before filling, we will fill the cell with the number of fatalities.</li>
    <li>If the number of fatalities is equal or lowe than the 'occupants' average before filling, we will fill the cell with the average.</li>
</ol>

In [14]:
occupants_avg = round(df['occupants'].mean()) #Assign the mean of 'occupants' to a variable

df_oc_col = df['occupants'] #Working on 'occupants' column
df_ft_col = df['fatalities'] #Working on 'fatalities' column

for i in range(0,len(df_oc_col)): #For every row:
    try:
        if math.isnan(df_oc_col[i]): #If the entry misses a value in 'occupants'
            if df_ft_col[i] > occupants_avg: #If 'fatalities' value in the entry is higher than the mean of occupants
                df_oc_col[i] = df_ft_col[i] #Assign the value in 'fatalities' to 'occupants' in the same row
            else:
                df_oc_col[i] = occupants_avg #Assign the value in the 'occupants' mean to 'occupants' in the same row
    except:
        None

In [15]:
df['occupants'] = df_oc_col
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4531 entries, 0 to 4530
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   weekday              4531 non-null   object 
 1   day                  4531 non-null   float64
 2   month                4531 non-null   object 
 3   year                 4531 non-null   int64  
 4   time                 4531 non-null   float64
 5   aircraft_type        4531 non-null   object 
 6   num_of_engines       4531 non-null   float64
 7   engine_type          4531 non-null   object 
 8   engine_model         4531 non-null   object 
 9   years_active         4298 non-null   float64
 10  airframe_hrs         2189 non-null   float64
 11  operator             4531 non-null   object 
 12  occupants            4531 non-null   float64
 13  accident_loc         4531 non-null   object 
 14  above_ocean          4531 non-null   int64  
 15  flight_phase         4531 non-null   o

<b>3) 'years_active'</b> and <b>'airframe_hrs'</b> - The method works as follows:<br>
<ul>
    <li>For entries that own values or both 'years_active' and 'airframe_hrs', we will add each value seperately to a different variable.</li>
    <li>Then we will divide the cumulative 'airframe_hrs' variable with the cumulative 'years_active' variable.</li>
    <li>The result represents the average airframe hours per year of activity for an aircraft.</li>
</ul>

In [16]:
df_ya_col = df['years_active'] #Working on 'years_active' column
df_ah_col = df['airframe_hrs'] #Working on 'airframe_hrs' column

cuml_ya = cuml_ah = 0 #Setting average variables for each column, only for entries that own both values.
num_of_both = 0 #Setting a variable to count how many entries own values for both columns

for i in range(0,len(df_ya_col)): #For every entry that owns both values, sums each to the respective variable
    if ((math.isnan(df_ya_col[i]) == False) & (math.isnan(df_ah_col[i]) == False)):
        cuml_ya = cuml_ya + df_ya_col[i]
        cuml_ah = cuml_ah + df_ah_col[i]
        num_of_both = num_of_both + 1 #Increments the 'num_of_both' variable to count how many rows own both values
        
cuml_ya = cuml_ya/num_of_both #Calculates average of 'years_active' of entries that also has value for 'airframe_hrs'
cuml_ah = round(cuml_ah/num_of_both) #Calculates average 'airframe_hours' of entries that also has value for 'years_active'

avg_ah_per_year = round(cuml_ah/cuml_ya)

<ul>
     <li>For entries that miss 'airframe_hrs' but don't miss 'years_active', we will assign the rounded value of the multiplication of the new average variable with the 'years_active' value.</li>
    <li>For entries that miss 'years_active' but don't miss 'airframe_hrs', we will assign the rounded value of the division of the 'airframe_hrs' value with the new average variable value.</li>
    <li>For entries that miss both columns, we will assign the average values we found for eavh column before we divided them to create a the combined variable</li>
</ul>

In [17]:
for i in range(0,len(df_ya_col)):
    if ((math.isnan(df_ya_col[i])) | (math.isnan(df_ah_col[i]))):
        if ((math.isnan(df_ya_col[i])) & (math.isnan(df_ah_col[i]))):
            df_ya_col[i] = cuml_ya
            df_ah_col[i] = cuml_ah
        elif (math.isnan(df_ah_col[i])):
            df_ah_col[i] = df_ya_col[i]*avg_ah_per_year
        elif (math.isnan(df_ya_col[i])):
            df_ya_col[i] = df_ah_col[i]/avg_ah_per_year

In [18]:
df['years_active'] = df_ya_col
df['airframe_hrs'] = df_ah_col
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4531 entries, 0 to 4530
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   weekday              4531 non-null   object 
 1   day                  4531 non-null   float64
 2   month                4531 non-null   object 
 3   year                 4531 non-null   int64  
 4   time                 4531 non-null   float64
 5   aircraft_type        4531 non-null   object 
 6   num_of_engines       4531 non-null   float64
 7   engine_type          4531 non-null   object 
 8   engine_model         4531 non-null   object 
 9   years_active         4531 non-null   float64
 10  airframe_hrs         4531 non-null   float64
 11  operator             4531 non-null   object 
 12  occupants            4531 non-null   float64
 13  accident_loc         4531 non-null   object 
 14  above_ocean          4531 non-null   int64  
 15  flight_phase         4531 non-null   o

<h3>Data Conversion to Numerical Values</h3>

We won't be able to apply a machine learning model on the dataframe without converting all values to numerical values. Let's understand which columns include string values, and we'll convert them accordingly

In [19]:
df.head()

,weekday,day,month,year,time,aircraft_type,num_of_engines,engine_type,engine_model,years_active,airframe_hrs,operator,occupants,accident_loc,above_ocean,flight_phase,damage,accident_latitude,accident_longtitude,fatalities
0,Saturday,2.0,August,1919,13.566667,Caproni Ca.48,3.0,piston,Liberty L-12,0.00,0.00,Caproni,14.0,Italy,0,ENR,Destroyed,45.396389,10.888056,14.0
1,Monday,11.0,August,1919,13.566667,Felixstowe Fury,5.0,piston,Rolls-Royce Eagle VIII,0.75,890.25,Royal Air Force - RAF,7.0,United Kingdom,0,ICL,Damaged beyond repair,51.941370,1.306789,1.0
2,Wednesday,18.0,August,1926,14.500000,Blériot 155,4.0,piston,Renault 8F,0.00,47.00,Air Union (France),15.0,United Kingdom,0,ENR,Damaged beyond repair,-51.174000,0.868000,4.0
3,Monday,22.0,August,1927,8.250000,Fokker F.VIII,2.0,piston,Bristol Jupiter,1.00,1187.00,KLM Royal Dutch Airlines,11.0,United Kingdom,0,ENR,Damaged beyond repair,51.250000,0.216000,1.0
4,Tuesday,19.0,March,1929,13.566667,Ford Tri-Motor,3.0,piston,Pratt & Whitney R-1340,0.00,0.00,Ford Motor Company,4.0,United States of America,0,LDG,Damaged beyond repair,42.300000,-83.216660,4.0


In [20]:
df.describe(include='all')

,weekday,day,month,year,time,aircraft_type,num_of_engines,engine_type,engine_model,years_active,airframe_hrs,operator,occupants,accident_loc,above_ocean,flight_phase,damage,accident_latitude,accident_longtitude,fatalities
count,4531,4531.000000,4531,4531.000000,4531.000000,4531,4531.000000,4531,4531,4531.000000,4531.000000,4531,4531.000000,4531,4531.000000,4531,4531,4531.000000,4531.000000,4531.000000
unique,7,NaN,12,NaN,NaN,309,NaN,3,123,NaN,NaN,2342,NaN,198,NaN,10,7,NaN,NaN,NaN
top,Wednesday,NaN,January,NaN,NaN,Douglas DC-3,NaN,jet,Pratt & Whitney Canada PT6,NaN,NaN,United States Air Force - USAF,NaN,United States of America,NaN,LDG,Damaged beyond repair,NaN,NaN,NaN
freq,715,NaN,444,NaN,NaN,342,NaN,2072,641,NaN,NaN,153,NaN,1141,NaN,1336,1836,NaN,NaN,NaN
mean,NaN,15.728537,NaN,1991.058707,13.430045,NaN,2.438314,NaN,NaN,16.221992,19258.685390,NaN,40.921651,NaN,0.016773,NaN,NaN,27.578006,-13.247964,13.646877
std,NaN,8.806055,NaN,20.693689,5.033081,NaN,0.896339,NaN,NaN,12.887761,16947.079201,NaN,60.553950,NaN,0.128435,NaN,NaN,24.694867,80.857836,33.506538
min,NaN,1.000000,NaN,1919.000000,0.016667,NaN,1.000000,NaN,NaN,0.000000,0.000000,NaN,0.000000,NaN,0.000000,NaN,NaN,-83.920000,-179.868000,0.000000
25%,NaN,8.000000,NaN,1976.000000,10.416667,NaN,2.000000,NaN,NaN,6.000000,5935.000000,NaN,4.000000,NaN,0.000000,NaN,NaN,10.765318,-80.259063,0.000000
50%,NaN,16.000000,NaN,1996.000000,13.566667,NaN,2.000000,NaN,NaN,14.583333,15431.000000,NaN,13.000000,NaN,0.000000,NaN,NaN,34.566500,-2.776605,2.000000
75%,NaN,23.000000,NaN,2008.000000,16.616667,NaN,3.000000,NaN,NaN,23.000000,27301.000000,NaN,50.500000,NaN,0.000000,NaN,NaN,45.271517,37.842913,9.000000


Out of our 20 columns, we have 9 columns which contain string values.<br>
We will prepare a list of possible values for each column, and assign a number to each unique value option.<br>
While all columns have less unique values than the amount of records, some have more value options than others (more aircraft types than the days of the week).<br>
That's why we will use two different ways to assign numbers to the columns' possible values.<br><br>

also worth mentioning is while during scraping we assigned a NaN value for columns that we found missing from the web page, some web pages explicitly mentioned "Unknown" for certain values, and those were scraped as valid values. During this step of string-to-numeric conversion, we'll convert this kind of values to NaN.

In [21]:
#==DEFINITVE REPLACING==#
#For columns with relatively low number of possible values, we will manually assign numbers to every value depending
#on which scale the column values' are on:

#weekday - Every weekday will be replaced with it's appropriate number depending on its position within the week:
weekdays_rep_map = {"Sunday" : 1, "Monday" : 2, "Tuesday" : 3, "Wednesday" : 4, "Thursday" : 5, "Friday" : 6, "Saturday" : 7}
df["weekday"].replace(weekdays_rep_map, inplace=True)

#weekday - Every month will be replaced with it's appropriate number depending on its position within the year:
months_rep_map = {"January" : 1, "February" : 2, "March" : 3, "April" : 4, "May" : 5, "June" : 6, "July" : 7, "August" : 8, "September" : 9, "October" : 10, "November" : 11, "December" : 12}
df["month"].replace(months_rep_map, inplace=True)

#engine_type - The least modern engine type will be numbered the lowest, and the most modern will be numbered the highest.
en_type_rep_map = {"piston" : 1, "turboprop" : 2, "jet" : 3}
df["engine_type"].replace(en_type_rep_map, inplace=True)

#flight_phase - Earlier flight phases will be numbered lower. UNK and CMB are assigned to accidents where the phase
#is not known. Therefore they'll be replaced with NaN.
phase_rep_map = {"UNK" : np.nan, "CMB" : np.nan, "STD" : 0, "PBT" : 1, "TXI" : 2, "TOF" : 3, "ICL" : 4, "ENR" : 5, "LDG" : 6, "APR" : 7, "MNV" : 8}
df["flight_phase"].replace(phase_rep_map, inplace=True)

#damage - Lower damage will have a lower number assigned to it. Unknown damage will be replaced with NaN.
damage_rep_map = {"Unknown" : np.nan, "None" : 0, "Minor" : 1, "Substantial" : 2, "Damaged beyond repair" : 3, "Destroyed" : 4, "Missing" : 5}
df["damage"].replace(damage_rep_map, inplace=True)

#==NON-DEFINITIVE REPLACING==
#For the remaining columns that we need to enumerate, there are dozens to hundreds possible values. In all columns, they are
#on a scale, so they can be numbered in any way. Therefore we number them depending on their alphabetical order:

#aircraft_type:
aircraft_type_labels = df["aircraft_type"].astype('category').cat.categories.to_list()
ac_type_rep_map={x:y for x,y in zip(aircraft_type_labels,list(range(0,len(aircraft_type_labels)+1)))}
df["aircraft_type"].replace(ac_type_rep_map, inplace=True)

#engine_model:
engine_model_labels = df["engine_model"].astype('category').cat.categories.to_list()
en_model_rep_map={x:y for x,y in zip(engine_model_labels,list(range(0,len(engine_model_labels)+1)))}
df["engine_model"].replace(en_model_rep_map, inplace=True)

#operator:
operator_labels = df["operator"].astype('category').cat.categories.to_list()
operator_rep_map={x:y for x,y in zip(operator_labels,list(range(0,len(operator_labels)+1)))}
df["operator"].replace(operator_rep_map, inplace=True)

#accident_loc:
location_labels = df["accident_loc"].astype('category').cat.categories.to_list()
loc_rep_map={x:y for x,y in zip(location_labels,list(range(0,len(location_labels)+1)))}
df["accident_loc"].replace(loc_rep_map, inplace=True)

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4531 entries, 0 to 4530
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   weekday              4531 non-null   int64  
 1   day                  4531 non-null   float64
 2   month                4531 non-null   int64  
 3   year                 4531 non-null   int64  
 4   time                 4531 non-null   float64
 5   aircraft_type        4531 non-null   int64  
 6   num_of_engines       4531 non-null   float64
 7   engine_type          4531 non-null   int64  
 8   engine_model         4531 non-null   int64  
 9   years_active         4531 non-null   float64
 10  airframe_hrs         4531 non-null   float64
 11  operator             4531 non-null   int64  
 12  occupants            4531 non-null   float64
 13  accident_loc         4531 non-null   int64  
 14  above_ocean          4531 non-null   int64  
 15  flight_phase         4495 non-null   f

Since our conversion created new NaN values, we will remove the rows with them to receive our final dataframe to use for the next steps

In [23]:
df.dropna(axis=0, subset=['damage', 'flight_phase'], inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4481 entries, 0 to 4530
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   weekday              4481 non-null   int64  
 1   day                  4481 non-null   float64
 2   month                4481 non-null   int64  
 3   year                 4481 non-null   int64  
 4   time                 4481 non-null   float64
 5   aircraft_type        4481 non-null   int64  
 6   num_of_engines       4481 non-null   float64
 7   engine_type          4481 non-null   int64  
 8   engine_model         4481 non-null   int64  
 9   years_active         4481 non-null   float64
 10  airframe_hrs         4481 non-null   float64
 11  operator             4481 non-null   int64  
 12  occupants            4481 non-null   float64
 13  accident_loc         4481 non-null   int64  
 14  above_ocean          4481 non-null   int64  
 15  flight_phase         4481 non-null   f

In [24]:
df.head() #We use the .head attribute again to sample the dataframe and see how it changed

,weekday,day,month,year,time,aircraft_type,num_of_engines,engine_type,engine_model,years_active,airframe_hrs,operator,occupants,accident_loc,above_ocean,flight_phase,damage,accident_latitude,accident_longtitude,fatalities
0,7,2.0,8,1919,13.566667,98,3.0,1,51,0.00,0.00,623,14.0,89,0,5.0,4.0,45.396389,10.888056,14.0
1,2,11.0,8,1919,13.566667,159,5.0,1,95,0.75,890.25,1722,7.0,187,0,4.0,3.0,51.941370,1.306789,1.0
2,4,18.0,8,1926,14.500000,51,4.0,1,88,0.00,47.00,314,15.0,187,0,5.0,3.0,-51.174000,0.868000,4.0
3,2,22.0,8,1927,8.250000,166,2.0,1,9,1.00,1187.00,1202,11.0,187,0,5.0,3.0,51.250000,0.216000,1.0
4,3,19.0,3,1929,13.566667,169,3.0,1,78,0.00,0.00,936,4.0,188,0,6.0,3.0,42.300000,-83.216660,4.0


To proceed to the next step in our project, we will export the dataframe into a .csv file, for later use:

In [25]:
df.to_csv("df_cleaned.csv", index=False)

The output file, named <b>"df_cleaned.csv"</b> can be found in our project folder in GitHub

We will extract the dictionary of the country names that was created in order to convert the 'accident_loc' column to numerical values, to a file - for further use in the next step:

In [229]:
loc_file = open("loc_dict.pkl", "wb")
pickle.dump(loc_rep_map, loc_file)
loc_file.close()